In [ ]:
import os
import uuid
import time
import threading
import cv2
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import messagebox

In [ ]:
IMAGES_PATH='Tensorflow/workspace/images/capturedImages'


In [ ]:
labels = ['hello','thanks']
no_img=5

In [ ]:
class RoundedToggle(tk.Frame):
    def __init__(self, master=None, on_toggle=None, **kwargs):
        super().__init__(master, **kwargs)
        self.is_on = False
        self.on_toggle = on_toggle

        self.canvas = tk.Canvas(self, width=60, height=30, bg=self["bg"], highlightthickness=0)
        self.canvas.pack()

        self.bg = self.canvas.create_oval(0, 0, 30, 30, fill="#e5e7eb", outline="")
        self.bg2 = self.canvas.create_oval(30, 0, 60, 30, fill="#e5e7eb", outline="")
        self.bg_rect = self.canvas.create_rectangle(15, 0, 45, 30, fill="#e5e7eb", outline="")

        self.knob = self.canvas.create_oval(3, 3, 27, 27, fill="white", outline="")

        self.canvas.bind("<Button-1>", self.toggle)

    def toggle(self, event=None):
        self.is_on = not self.is_on
        if self.is_on:
            self.canvas.itemconfig(self.bg, fill="#4ade80")
            self.canvas.itemconfig(self.bg2, fill="#4ade80")
            self.canvas.itemconfig(self.bg_rect, fill="#4ade80")
            self.canvas.coords(self.knob, 33, 3, 57, 27)
        else:
            self.canvas.itemconfig(self.bg, fill="#e5e7eb")
            self.canvas.itemconfig(self.bg2, fill="#e5e7eb")
            self.canvas.itemconfig(self.bg_rect, fill="#e5e7eb")
            self.canvas.coords(self.knob, 3, 3, 27, 27)

        if self.on_toggle:
            self.on_toggle(self.is_on)


def toggle_voice_feedback(state):
    if state:
        voice_toggle_label.config(text="Voice Feedback Enabled")
    else:
        voice_toggle_label.config(text="Voice Feedback Disabled")


def update_camera():
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 1)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)
        camera_label.imgtk = imgtk
        camera_label.configure(image=imgtk)
    camera_label.after(10, update_camera)


def capture_images(label):
    os.makedirs(f'{IMAGES_PATH}{label}', exist_ok=True)
    print(f"\nGet ready to capture images for: {label}")
    time.sleep(1)

    for _ in range(10):  
        cap.read()
        time.sleep(0.05)

    print("Starting 5-second capture...")
    start_time = time.time()
    while time.time() - start_time <= 5:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        frame = cv2.flip(frame, 1)
        imagename = os.path.join(IMAGES_PATH, label, f'{label}.{str(uuid.uuid1())}.jpg')
        cv2.imwrite(imagename, frame)
        print(f"Saved {imagename}")
        time.sleep(0.3)

    messagebox.showinfo("Done Caputring")

def start_capture_thread():
    def on_select():
        selected_label = label_var.get()
        if selected_label in labels:
            popup.destroy()
            threading.Thread(target=capture_images, args=(selected_label,), daemon=True).start()

    popup = tk.Toplevel(root)
    popup.title("Select Label")
    popup.geometry("300x120")
    popup.configure(bg="#f6f9fc")

    label_prompt = tk.Label(popup, text="Select label for capture:", bg="#f6f9fc", font=("Segoe UI", 10))
    label_prompt.pack(pady=(15, 5))

    label_var = tk.StringVar(popup)
    label_var.set(labels[0])  

    dropdown = tk.OptionMenu(popup, label_var, *labels)
    dropdown.pack(pady=5)

    confirm_btn = tk.Button(popup, text="Start Capture", command=on_select, bg="#60a5fa", fg="white",
                            font=("Segoe UI", 10, "bold"))
    confirm_btn.pack(pady=(5, 10))



root = tk.Tk()
root.title("Hand Gesture Recognizer")
root.geometry("1000x700")
root.configure(bg="#f6f9fc")

title = tk.Label(root, text="Hand Gesture Recognizer", font=("Segoe UI", 24, "bold"), bg="#f6f9fc")
title.pack(pady=(20, 0))

subtitle = tk.Label(root, text="Display hand gestures as text and hear them spoken aloud", font=("Segoe UI", 12),
                    fg="#6b7280", bg="#f6f9fc")
subtitle.pack(pady=(5, 20))

main_frame = tk.Frame(root, bg="#f6f9fc")
main_frame.pack(fill="both", expand=True, padx=20, pady=10)

camera_frame = tk.Frame(main_frame, bg="#f6f9fc")
camera_frame.pack(side="left", fill="both", expand=True, padx=(0, 10))

camera_box = tk.Frame(camera_frame, bg="#fee2e2", width=640, height=400,
                      highlightbackground="#60a5fa", highlightcolor="#60a5fa", highlightthickness=2)
camera_box.pack(fill="both", expand=True)
camera_box.pack_propagate(False)

camera_label = tk.Label(camera_box, text="Camera initializing...", bg="#fee2e2", fg="#b91c1c",
                        font=("Segoe UI", 12))
camera_label.pack(expand=True)

detection_label = tk.Label(camera_frame, text="● Detection Active", fg="#22c55e", bg="#f6f9fc",
                           font=("Segoe UI", 10, "bold"))
detection_label.pack(pady=5)

gesture_frame = tk.Frame(main_frame, width=300, height=400, bg="white", bd=1, relief="solid")
gesture_frame.pack(side="left", fill="y", padx=(10, 0))
gesture_frame.pack_propagate(False)

gesture_title = tk.Label(gesture_frame, text="Recognized Gesture", font=("Segoe UI", 16, "bold"), bg="white")
gesture_title.pack(pady=(20, 10))

gesture_icon = tk.Label(gesture_frame, text="📷", font=("Segoe UI", 32), bg="white", fg="#cbd5e1")
gesture_icon.pack(pady=10)

gesture_text = tk.Label(gesture_frame, text="No gesture detected", font=("Segoe UI", 12), fg="#6b7280", bg="white")
gesture_text.pack()

gesture_subtext = tk.Label(gesture_frame, text="Show your hand to the camera", font=("Segoe UI", 10),
                           fg="#9ca3af", bg="white")
gesture_subtext.pack(pady=(5, 0))

bottom_frame = tk.Frame(root, bg="#f6f9fc")
bottom_frame.pack(side="bottom", fill="x", pady=10, padx=20)

voice_toggle = RoundedToggle(bottom_frame, on_toggle=toggle_voice_feedback, bg="#f6f9fc")
voice_toggle.pack(side="left", padx=(0, 5))

voice_toggle_label = tk.Label(bottom_frame, text="Voice Feedback Disabled", font=("Segoe UI", 10),
                              fg="#374151", bg="#f6f9fc")
voice_toggle_label.pack(side="left")

capture_button = tk.Button(bottom_frame, text="Start Capture", font=("Segoe UI", 10, "bold"), bg="#60a5fa",
                           fg="white", command=start_capture_thread)
capture_button.pack(side="right")

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

update_camera()  
root.mainloop()

cap.release()
cv2.destroyAllWindows()